<a href="https://colab.research.google.com/github/ranjithreddyy/collab_books/blob/main/pyspark_for_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

download kaggle json from kaggle webite 
https://www.kaggle.com/code/ranjithwxyz/pyspark-for-data-science/edit


In [ ]:
from google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic
!kaggle datasets download -d tientd95/movierating
!unzip titanic.zip
!unzip movierating.zip

Saving kaggle.json to kaggle.json
  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 23.4MB/s]
  0% 0.00/383k [00:00<?, ?B/s]
100% 383k/383k [00:00<00:00, 95.3MB/s]
Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
Archive:  movierating.zip
  inflating: movie_ratings_df.csv    


Spark aims at light_speed, ease-of-use, extensibility in very large data system. Companies with large data center often deploy model on Spark in order to get the robust and scalable benefit on production level.
Spark has introduced high-level APIs in Java, Scala, Python, and R. 
I use Pyspark for the familiarity. 

![Spark-in-Big-Data-Pipeline.png](attachment:Spark-in-Big-Data-Pipeline.png)

This kernel focusing on <font color="red"><b>Data processing</b></font> in  PySparkSQL/DataFrame and <font color="red"><b>modelling</b></font> with Spark Machine learning (Mlib) library.


Thank you for reading my work and hoping you like it

have to download kaggle api from kaggle website to download the files from kaggle website 


pip install pipenv<a class="anchor" id="0.1"></a>

# **Table of Contents**


1.	[Movie Recommendation with Pyspark](#1)
2.	[Classification model in Pyspark](#2)
    - 2.1	[EDA in Pyspark](#2.1)
    - 2.2   [Machine learning in Pyspark](#2.2)

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=772b6d273920c57b306bb5b1e44c24c2a064b5bdb0ef4c611a7dd5a779471b6f
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
import os

from pyspark.sql import SparkSession 
from pyspark.ml  import Pipeline     
from pyspark.sql import SQLContext  
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

# **1. Movie Recommendation with Pyspark** <a class="anchor" id="1"></a>

[Go back to table of contents](#0.1)

Firstly, We need to create a spark container by calling SparkSession. This step is necessary before doing anything

In [ ]:
# Create SparkSession
spark = SparkSession.builder.appName('recommender_system').getOrCreate()

### Pyspark.toPandas() method allow us to return SparkDataFrame as Pandas table display

In [ ]:
# load the dataset and create sprk dataframe
df=spark.read.csv('/content/movie_ratings_df.csv',inferSchema=True,header=True)
# Using limit(), or select() or show() to view the data. I often use limit()
# Using toPandas() method to return Pyspark DataFrame as Pandas table
df.show(3)

+------+------------+------+
|userId|       title|rating|
+------+------------+------+
|   196|Kolya (1996)|     3|
|    63|Kolya (1996)|     3|
|   226|Kolya (1996)|     5|
+------+------------+------+
only showing top 3 rows



## Our task: given a user, we predict and return a list of movies recommendation for that user to watch.

### We use: **printSchema()** to quick overview of features datatype

In [ ]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



### As we can see, the title column is stored as string type. To work with pyspark Mlib library, we need to convert string type to numeric values

In [ ]:
from pyspark.ml.feature import StringIndexer, IndexToString
stringIndexer = StringIndexer(inputCol='title', outputCol='title_new')
# Applying stringindexer object on dataframe movie title column
model = stringIndexer.fit(df)
#creating new dataframe with transformed values
indexed = model.transform(df)
# #validate the numerical title values
indexed.toPandas()

,userId,title,rating,title_new
0,196,Kolya (1996),3,287.0
1,63,Kolya (1996),3,287.0
2,226,Kolya (1996),5,287.0
3,154,Kolya (1996),3,287.0
4,306,Kolya (1996),5,287.0
...,...,...,...,...
99995,840,Mamma Roma (1962),4,1597.0
99996,655,"Eighth Day, The (1996)",3,1555.0
99997,655,Girls Town (1996),3,1568.0
99998,655,"Silence of the Palace, The (Saimt el Qusur) (1...",3,1628.0


### We use Alternating least squares (ALS) algorithm in Pyspark Ml library for recommendation. To read more, you can visit at https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

In [ ]:
# split the data into training and test datatset
train, test = indexed.randomSplit([0.75,0.25])

In [ ]:
from pyspark.ml.recommendation import ALS

#Training the recommender model using train datatset
rec=ALS( maxIter=10
        ,regParam=0.01
        ,userCol='userId'
        ,itemCol='title_new'
        ,ratingCol='rating'
        ,nonnegative=True
        ,coldStartStrategy="drop")

#fit the model on train set
rec_model=rec.fit(train)

#making predictions on test set 
predicted_ratings=rec_model.transform(test)
predicted_ratings.limit(5).toPandas()

,userId,title,rating,title_new,prediction
0,148,2001: A Space Odyssey (1968),5,59.0,4.881881
1,148,"Abyss, The (1989)",4,200.0,3.470735
2,148,Being There (1979),5,290.0,7.425135
3,148,Dr. Strangelove or: How I Learned to Stop Worr...,5,123.0,7.014516
4,148,Four Weddings and a Funeral (1994),5,66.0,3.942170


### Evaluate the training

In [ ]:
# Importing Regression Evaluator to measure RMSE
from pyspark.ml.evaluation import RegressionEvaluator
# create Regressor evaluator object for measuring accuracy
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')
# apply the RE on predictions dataframe to calculate RMSE
rmse=evaluator.evaluate(predicted_ratings)
# print RMSE error
print(rmse)

1.0237301475577005


### After training, now is the time to recommend top movies which user might like 

In [ ]:
# First we need to create dataset of all distinct movies 
unique_movies=indexed.select('title_new').distinct()

#create function to recommend top 'n' movies to any particular user
def top_movies(user_id,n):
    """
    This function returns the top 'n' movies that user has not seen yet but might like 
    
    """
    #assigning alias name 'a' to unique movies df
    a = unique_movies.alias('a')
    
    #creating another dataframe which contains already watched movie by active user 
    watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new')
    
    #assigning alias name 'b' to watched movies df
    b=watched_movies.alias('b')
    
    #joining both tables on left join 
    total_movies = a.join(b, a.title_new == b.title_new,how='left')
    
    #selecting movies which active user is yet to rate or watch
    remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
    
    
    #adding new column of user_Id of active useer to remaining movies df 
    remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
    
    
    #making recommendations using ALS recommender model and selecting only top 'n' movies
    recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False).limit(n)
    
    
    #adding columns of movie titles in recommendations
    movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)
    final_recommendations=movie_title.transform(recommendations)
    
    #return the recommendations to active user
    return final_recommendations.show(n,False)

In [ ]:
# Test: recommend 5 movies for user of id=60
top_movies(60,5)

# **2. Classification model in Pyspark** <a class="anchor" id="2"></a>

[Go back to table of contents](#0.1)

We using Titanic dataset for familiar playing with data based on Pyspark DataFrame

## **2.1 EDA in Pyspark** <a class="anchor" id="2.1"></a>

[Go back to table of contents](#0.1)

In [ ]:
# Firstly we create sparkSession (like a container)
spark = SparkSession.builder.appName('Play with pyspark ML on titatic_dataset').getOrCreate()
# After creating spark, we use spark.read.csv to read dataset, like pandas.read_csv
df = spark.read.csv('../input/titanic/train.csv',header = 'True',inferSchema='True')

In [ ]:
df.limit(3).toPandas()

In [ ]:
df.printSchema()

### For visualization,before using visual library (matplotlib, seaborn, ..)we need to convert SparkDataframe to PandasDataFrame 

In [ ]:
pandas_df = df.toPandas()

plt.figure(figsize=(10,5))
plt.title('Age distribution among all Pasengers')
sns.distplot(pandas_df['Age']);

## Checking null values in Pyspark

* isnan() is a function of the pysparq.sql.function package, we have to set which column we want to use as an argument of the function. 
* isNull()" belongs to pyspark.sql.Column package, to check the null status of a column

To check null in Pyspark, we use both function above

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
# Because Cabin column has a lot missing value ( > 50%) so we can remove it
df = df.drop("Cabin")

To replace these NaN values, we can assign them the mean age of the dataset.But the problem is, there were many people with many different ages. 

In 'Name' feature, we can see that the names have a salutation like Mr or Mrs. Thus we can assign the mean values of Mr and Mrs to the respective groups

**Using the Regex ""[A-Za-z]+)." we extract the initials from the Name. It looks for strings which lie between A-Z or a-z and followed by a .(dot).**

In [ ]:
df = df.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))
df.limit(3).toPandas()

In [ ]:
# There are some misspelled Initials like Mlle or Mme that stand for Miss. we will replace them with Miss and same thing for other values.
df = df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                        ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

# Checking the average age by Initials
df.groupby('Initial').avg('Age').collect()

In [ ]:
# Assigning missing values in age feature based on average age of Initials
df = df.withColumn("Age",when((df["Initial"] == "Miss")   & (df["Age"].isNull()), 22).otherwise(df["Age"]))
df = df.withColumn("Age",when((df["Initial"] == "Other")  & (df["Age"].isNull()), 46).otherwise(df["Age"]))
df = df.withColumn("Age",when((df["Initial"] == "Master") & (df["Age"].isNull()), 5).otherwise( df["Age"]))
df = df.withColumn("Age",when((df["Initial"] == "Mr")     & (df["Age"].isNull()), 33).otherwise(df["Age"]))
df = df.withColumn("Age",when((df["Initial"] == "Mrs")    & (df["Age"].isNull()), 36).otherwise(df["Age"]))

In [ ]:
# Embarked columns has two missing values, let check it
df.groupBy("Embarked").count().show()

### We using PysparkDataFrame.na.fill() to fill a value to specific column

In [ ]:
# As we can see, Majority Passengers boarded from "S". We can assign missing value with "S"
df = df.na.fill({"Embarked" : 'S'})
# Check again the missing value status
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

#### After handling missing values, we do some simple feature engineering
#### in Feature engineering, we can use Pyspark multiple condition with syntax: "When otherwise"
#### To learn more about multiple condition in pyspark, you can visit at https://sparkbyexamples.com/spark/spark-case-when-otherwise-example/

In [ ]:
df = df.withColumn("Family_Size",col('SibSp')+col('Parch')) # Create new column: Family_size
df = df.withColumn('Alone',lit(0))  # Create new column: Alone and assign 0 default value to Alone column
df = df.withColumn("Alone",when(df["Family_Size"] == 0, 1).otherwise(df["Alone"]))

# converting Sex, Embarked & Initial columns from string to number using StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in ["Sex","Embarked","Initial"]]
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

In [ ]:
df.limit(3).toPandas()

In [ ]:
# Now we drop columns that not needed for modelling
df = df.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")

# Before modelling in Pyspark, we need to put all features to Vector using Pyspark VectorAssembler
feature = VectorAssembler(inputCols = df.columns[1:],outputCol="features")
feature_vector= feature.transform(df)
feature_vector.limit(3).toPandas()

In [ ]:
# for data split in pyspark, we can use df.randomSplit()
(train_df, test_df) = feature_vector.randomSplit([0.8, 0.2],seed = 11)

In [ ]:
train_df.printSchema()

## **2.2 Machine learning in pyspark** <a class="anchor" id="2.2"></a>

[Go back to table of contents](#0.1)


### Just for practising, I'll use some of learning algorithm

1. LogisticRegression
 
1. RandomForestClassifier

1. Gradient-boosted tree classifier 

In [ ]:
# Select features column for features training and 'Survived' as label to predict
titanic_df = feature_vector.select(['features','Survived'])
# Split the dataset to train_df and test_df
train_df,test_df = titanic_df.randomSplit([0.75,0.25])

## Logistic regression

In [ ]:
# LOAD PYSPARK LIBRARIES
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from sklearn.metrics import roc_curve,auc

# DEFINE ALGORITHM 
lr = LogisticRegression(labelCol="Survived")

# DEFINE GRID PARAMETERS
paramGrid = ParamGridBuilder().addGrid(lr.regParam, (0.01, 0.1))\
                              .addGrid(lr.maxIter, (5, 10))\
                              .addGrid(lr.tol, (1e-4, 1e-5))\
                              .addGrid(lr.elasticNetParam, (0.25,0.75))\
                              .build()

# DEFINE CROSS VALIDATION WITH PARAMETERS
tvs = TrainValidationSplit( estimator=lr
                           ,estimatorParamMaps=paramGrid
                           ,evaluator=MulticlassClassificationEvaluator(labelCol='Survived')
                           ,trainRatio=0.8)
model = tvs.fit(train_df)
model_predictions= model.transform(test_df)

print('Accuracy: ', MulticlassClassificationEvaluator(labelCol='Survived',metricName='accuracy').evaluate(model_predictions))
print('Precision: ',MulticlassClassificationEvaluator(labelCol='Survived',metricName='weightedPrecision').evaluate(model_predictions))


### Evaluating ROC metric

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric
from pyspark import SparkContext
sc =SparkContext.getOrCreate()   # We need to create SparkContext
results = model_predictions.select(['probability', 'Survived'])
 
## prepare score-label set
results_collect = results.collect()
results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]
scoreAndLabels = sc.parallelize(results_list)
 
metrics = metric(scoreAndLabels)
print("The ROC score is : ", metrics.areaUnderROC)

### Visualizing AUC metrics

In [ ]:
from sklearn.metrics import roc_curve, auc
 
fpr = dict()
tpr = dict()
roc_auc = dict()
 
y_test = [i[1] for i in results_list]
y_score = [i[0] for i in results_list]
 
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
 
%matplotlib inline
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Area under the ROC Curve')
plt.legend(loc="lower right")
plt.show()

## Random Forest

In [ ]:

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
rf = RandomForestClassifier(labelCol='Survived')
paramGrid = ParamGridBuilder()\
                                .addGrid(rf.maxDepth, [5, 10, 20]) \
                                .addGrid(rf.maxBins, [20, 32, 50]) \
                                .addGrid(rf.numTrees, [20, 40, 60 ]) \
                                .addGrid(rf.impurity, ["gini", "entropy"]) \
                                .addGrid(rf.minInstancesPerNode, [1, 5, 10]) \
                                .build()


tvs = TrainValidationSplit( estimator=rf
                           ,estimatorParamMaps=paramGrid
                           ,evaluator=MulticlassClassificationEvaluator(labelCol='Survived')
                           ,trainRatio=0.8)
model = tvs.fit(train_df)
model_predictions= model.transform(test_df)


print('Accuracy: ', MulticlassClassificationEvaluator(labelCol='Survived',metricName='accuracy').evaluate(model_predictions))
print('Precision: ',MulticlassClassificationEvaluator(labelCol='Survived',metricName='weightedPrecision').evaluate(model_predictions))

### Evaluating ROC metrics

In [ ]:
sc =SparkContext.getOrCreate()   # We need to create SparkContext
results = model_predictions.select(['probability', 'Survived'])
 
## prepare score-label set
results_collect = results.collect()
results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]
scoreAndLabels = sc.parallelize(results_list)
 
metrics = metric(scoreAndLabels)
print("The ROC score is : ", metrics.areaUnderROC)

### Visualizing AUC metrics

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
 
y_test = [i[1] for i in results_list]
y_score = [i[0] for i in results_list]
 
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
 
%matplotlib inline
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Area under the ROC Curve')
plt.legend(loc="lower right")
plt.show()

## Gradient Boosted Tree

If you want to run the below part, just uncomment it. I dit it in order to reduce the time of committing the kernel.

In [ ]:

# from pyspark.ml.classification import GBTClassifier
# gbt = GBTClassifier(labelCol="Survived")
# paramGrid = ParamGridBuilder()\
#                             .addGrid(gbt.maxDepth, [5, 10, 20]) \
#                             .addGrid(gbt.maxBins, [20, 32, 50]) \
#                             .addGrid(gbt.maxIter, [10, 20, 30]) \
#                             .addGrid(gbt.minInstancesPerNode, [1, 5, 10]) \
#                             .build()
    
# tvs = TrainValidationSplit(estimator=gbt
#                            , estimatorParamMaps=paramGrid
#                             ,evaluator=MulticlassClassificationEvaluator(labelCol='Survived')
#                             ,trainRatio=0.8)
# model = tvs.fit(train_df)
# model_predictions= model.transform(test_df)
# print('Accuracy: ', MulticlassClassificationEvaluator(labelCol='Survived',metricName='accuracy').evaluate(model_predictions))
# print('Precision: ',MulticlassClassificationEvaluator(labelCol='Survived',metricName='weightedPrecision').evaluate(model_predictions))


### Evaluating ROC metric

In [ ]:
# sc =SparkContext.getOrCreate()   # We need to create SparkContext
# results = model_predictions.select(['probability', 'Survived'])
 
# ## prepare score-label set
# results_collect = results.collect()
# results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]
# scoreAndLabels = sc.parallelize(results_list)
 
# metrics = metric(scoreAndLabels)
# print("The ROC score is: ", metrics.areaUnderROC)

### Visualizing AUC metrics

In [ ]:
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
 
# y_test = [i[1] for i in results_list]
# y_score = [i[0] for i in results_list]
 
# fpr, tpr, _ = roc_curve(y_test, y_score)
# roc_auc = auc(fpr, tpr)
 
# %matplotlib inline
# plt.figure()
# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Area under the ROC Curve')
# plt.legend(loc="lower right")
# plt.show()

I hope you find this notebook beneficial and enjoyable